# Full Training Pipeline - Production Workflow

This notebook provides a complete, production-ready training workflow for the Sequence framework.

**Features:**
- Requirements installation
- Multi-pair sequential training
- Real data loading from prepared CSV files
- Train/validation/test splitting
- Multi-GPU training support
- Automatic Mixed Precision (AMP)
- Early stopping and learning rate scheduling
- Supervised learning (classification)
- Reinforcement learning (SAC)
- Checkpoint management
- Comprehensive evaluation and visualization

## 0. Mounting Google Drive

In [4]:
#!/usr/bin/env python3
#Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted!")

ValueError: mount failed

### 2. Repository Extraction

In [ ]:
import zipfile
from pathlib import Path

zip_path = Path('/content/drive/MyDrive/Sequence.zip')
extract_to = Path('/content/Sequence')

if not zip_path.exists():
    print(f"❌ ERROR: {zip_path} not found")
    print("Upload Sequence.zip to your Google Drive root (MyDrive)")
else:
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content')

    # Handle possible directory naming
    extracted = [d for d in Path('/content').iterdir() if d.is_dir() and 'Sequence' in d.name]
    if extracted and extracted[0] != extract_to:
        extracted[0].rename(extract_to)

    print("✓ Repository extracted!")

### 3. Fix Path Configuration

In [ ]:
import sys
from pathlib import Path

# UPDATED FOR COLAB
ROOT = Path('/content/Sequence')  # Changed from /Volumes/Containers/Sequence
sys.path.insert(0, str(ROOT))
sys.path.insert(0, str(ROOT / 'run'))

# Create necessary directories
(ROOT / 'data' / 'data').mkdir(parents=True, exist_ok=True)
(ROOT / 'data' / 'raw').mkdir(parents=True, exist_ok=True)
(ROOT / 'models' / 'checkpoints').mkdir(parents=True, exist_ok=True)

print(f"✓ Root: {ROOT}")

### 4. Install Dependencies

In [ ]:
# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
!pip install -q numpy pandas matplotlib seaborn scikit-learn tqdm
!pip install -q transformers backtesting ta histdata

print("✓ All requirements installed!")

### 5. **CONFIGURATION** 

In [ ]:
# =============================================================================
# MULTI-PAIR CONFIGURATION
# =============================================================================
# Training will run sequentially for each pair in the list
PAIRS = [
    "eurusd","eurgbp","eurjpy","eurchf",
    "euraud","eurcad","eurnzd","gbpusd", 
    "gbpjpy","gbpchf","gbpcad","gbpaud",
    "gbpnzd","usdjpy","usdchf","usdcad",
    "audusd","audjpy","audcad","audchf",
    "audnzd","nzdusd","nzdjpy","nzdcad",
    "nzdchf","cadchf","cadjpy","chfjpy",
    "usdbrl","usdrub","usdinr","usdcny",
    "usdzar","usdtry"
]  # Add more pairs as needed

# Training mode: 'supervised' or 'sac'
TRAINING_MODE = "supervised"  # Options: 'supervised', 'sac'

# Data configuration
T_IN = 120  # Input sequence length
T_OUT = 10  # Prediction horizon

# =============================================================================
# SUPERVISED LEARNING CONFIGURATION
# =============================================================================
SUPERVISED_CONFIG = {
    # Model architecture
    "hidden_size_lstm": 128,
    "num_layers_lstm": 2,
    "cnn_num_filters": 64,
    "attention_dim": 128,
    "dropout": 0.2,
    "num_classes": 3,  # 3 for classification (down/neutral/up)
    # Training hyperparameters
    "epochs": 50,
    "batch_size": 64,
    "learning_rate": 1e-3,
    "weight_decay": 1e-5,
    "grad_clip": 1.0,
    "use_amp": False,
    # Data splitting
    "train_ratio": 0.7,
    "val_ratio": 0.15,
    "test_ratio": 0.15,
    # Checkpointing
    "checkpoint_dir": "models/checkpoints",
    "early_stop_patience": 5,
    "lr_scheduler_patience": 3,
    "lr_scheduler_factor": 0.5,
}

# =============================================================================
# SAC (REINFORCEMENT LEARNING) CONFIGURATION
# =============================================================================
SAC_CONFIG = {
    # Agent hyperparameters
    "hidden_dim": 256,
    "learning_rate": 3e-4,
    "gamma": 0.99,  # Discount factor
    "tau": 0.005,  # Soft update coefficient
    "alpha": 0.2,  # Entropy temperature (if not auto-tuning)
    "auto_entropy_tuning": True,
    # Training configuration
    "total_steps": 100000,
    "batch_size": 256,
    "replay_buffer_size": 100000,
    "warmup_steps": 1000,  # Random exploration steps
    "update_interval": 1,  # Update every N steps
    "eval_interval": 5000,  # Evaluate every N steps
    "eval_episodes": 10,
    # Environment configuration
    "initial_cash": 50000.0,
    "time_horizon": 100,
    "commission_pct": 0.0001,  # 0.01%
    "spread_bps": 1.0,  # 1 basis point
    "reward_type": "incremental_pnl",  # Options: 'portfolio_value', 'incremental_pnl', 'sharpe', 'cost_aware'
    "reward_scaling": 1.0,
    # Checkpointing
    "checkpoint_dir": "models/sac_checkpoints",
    "save_interval": 10000,
}

print(f"Training mode: {TRAINING_MODE.upper()}")
print(f"Pairs to train: {PAIRS}")
print(f"Total pairs: {len(PAIRS)}")

### 6. ***GPU CHECK***

In [ ]:
# Check GPU availability
print("="*60)
print("GPU Environment Check")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print("\nGPU Details:")
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"  GPU {i}: {props.name}")
        print(f"    Memory: {props.total_memory / 1024**3:.2f} GB")
        print(f"    Compute Capability: {props.major}.{props.minor}")
    
    print("\nCurrent GPU Utilization:")
    print_gpu_utilization()
else:
    print("\n⚠️  No GPU detected. Training will use CPU.")
    print("For best performance, run on a GPU-enabled environment.")

print("="*60)

### 7. **DATA COLLECTION FROM HISTDATA** 

In [ ]:
from histdata.api import download_hist_data
from pathlib import Path

# Configuration (update these as needed)
PAIRS_TO_DOWNLOAD = ["eurusd","eurgbp","eurjpy","eurchf",
    "euraud","eurcad","eurnzd","gbpusd", 
    "gbpjpy","gbpchf","gbpcad","gbpaud",
    "gbpnzd","usdjpy","usdchf","usdcad",
    "audusd","audjpy","audcad","audchf",
    "audnzd","nzdusd","nzdjpy","nzdcad",
    "nzdchf","cadchf","cadjpy","chfjpy",
    "usdbrl","usdrub","usdinr","usdcny",
    "usdzar","usdtry"]  # Start with 2-3 pairs
YEARS_TO_DOWNLOAD = ['2010','2011','2012','2013','2014','2015',
                     '2016','2017','2018','2019','2020','2021','2022','2023','2024','2025']
print("📥 Downloading Historical FX Data from HistData.com")
print("="*70)
print(f"Pairs: {', '.join([p.upper() for p in PAIRS_TO_DOWNLOAD])}")
print(f"Years: {', '.join(YEARS_TO_DOWNLOAD)}")
print("="*70 + "\n")

raw_data_dir = ROOT / 'data' / 'raw'
raw_data_dir.mkdir(parents=True, exist_ok=True)

download_stats = {'successful': 0, 'failed': 0}

for pair in PAIRS_TO_DOWNLOAD:
    print(f"\n{'─'*70}")
    print(f"📊 Pair: {pair.upper()}")
    print(f"{'─'*70}")

    pair_dir = raw_data_dir / pair
    pair_dir.mkdir(parents=True, exist_ok=True)

    for year in YEARS_TO_DOWNLOAD:
        try:
            print(f"  {year}... ", end='', flush=True)

            # Try downloading full year first
            try:
                result = download_hist_data(
                    year=int(year),
                    pair=pair,
                    output_directory=str(pair_dir),
                    verbose=False
                )
                print(f"✓ {result}")
                download_stats['successful'] += 1

            except AssertionError:
                # Download month-by-month if full year not available
                print("(month-by-month)")
                months_ok = 0
                for month in range(1, 13):
                    try:
                        download_hist_data(
                            year=int(year),
                            month=month,
                            pair=pair,
                            output_directory=str(pair_dir),
                            verbose=False
                        )
                        months_ok += 1
                        download_stats['successful'] += 1
                    except:
                        pass
                print(f"    ✓ {months_ok} months downloaded")

        except Exception as e:
            print(f"✗ {str(e)[:50]}")
            download_stats['failed'] += 1

print(f"\n{'='*70}")
print(f"✓ Successful: {download_stats['successful']}")
print(f"✗ Failed: {download_stats['failed']}")
print(f"✓ Data collection complete!")
print(f"{'='*70}")

### 8. ***DATA PREPARATION***

In [ ]:
import subprocess

print("Starting data preparation pipeline...\\n")

for pair in PAIRS_TO_DOWNLOAD:  # Changed from PAIRS_TO_COLLECT
    print(f"\\nProcessing {pair.upper()}...")

    cmd = [
        'python', str(ROOT / 'data' / 'prepare_dataset.py'),
        '--pairs', pair,
        '--t-in', str(T_IN),
        '--t-out', str(T_OUT),
        '--task-type', TASK_TYPE,
    ]

    # Add optional flags
    if USE_INTRINSIC_TIME:
        cmd.append('--intrinsic-time')
    if INCLUDE_SENTIMENT:
        cmd.append('--include-sentiment')

    result = subprocess.run(cmd, capture_output=True, text=True, cwd=str(ROOT))

    if result.returncode == 0:
        print(f"  ✓ {pair.upper()} prepared")
    else:
        print(f"  ✗ Failed: {result.stderr}")

print("\\n✓ Data preparation complete!")

### 9. ***SETUP AND IMPORTS***

In [ ]:
# System setup
import sys
from pathlib import Path

# Add project root to path
ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(ROOT))
sys.path.insert(0, str(ROOT / 'run'))

print(f"Project root: {ROOT}")

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from IPython.display import clear_output

# Project imports
from config.config import (
    AssetConfig, AssetClass, DataConfig, 
    FeatureConfig, ModelConfig, TrainingConfig
)
from utils.multi_gpu import setup_multi_gpu, print_gpu_utilization, get_gpu_memory_info
from train.features.agent_features import build_feature_frame

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

### 10. ***Helper Functions***

In [ ]:
def get_unwrapped_model(model):
    """Get the underlying model from DataParallel wrapper."""
    return model.module if isinstance(model, nn.DataParallel) else model


def format_time(seconds):
    """Format seconds into human-readable time."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    
    if hours > 0:
        return f"{hours}h {minutes}m {secs}s"
    elif minutes > 0:
        return f"{minutes}m {secs}s"
    else:
        return f"{secs}s"


class SequenceDataset(Dataset):
    """PyTorch Dataset for time series sequences."""
    
    def __init__(self, features, targets, t_in):
        self.features = torch.FloatTensor(features)
        self.targets = torch.LongTensor(targets) if targets.dtype == np.int64 else torch.FloatTensor(targets)
        self.t_in = t_in
    
    def __len__(self):
        return len(self.features) - self.t_in
    
    def __getitem__(self, idx):
        x = self.features[idx:idx + self.t_in]
        y = self.targets[idx + self.t_in]
        return x, y

### 11. ***Supervised Learning Functions***

In [ ]:
def train_epoch_supervised(model, loader, criterion, optimizer, device, scaler=None, grad_clip=None):
    """Train for one epoch (supervised learning)."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for x, y in tqdm(loader, desc="Training", leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(x)
                loss = criterion(outputs, y)
            scaler.scale(loss).backward()
            if grad_clip:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()
    
    return total_loss / len(loader), 100. * correct / total


def validate_supervised(model, loader, criterion, device):
    """Validate the model (supervised learning)."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in tqdm(loader, desc="Validating", leave=False):
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()
    
    return total_loss / len(loader), 100. * correct / total


def train_supervised(pair, config):
    """Complete supervised training pipeline for a single pair."""
    print(f"\n{'='*80}")
    print(f"SUPERVISED TRAINING: {pair.upper()}")
    print(f"{'='*80}\n")
    
    import time
    start_time = time.time()
    
    # Load data
    data_path = ROOT / 'data' / 'data' / pair / f'{pair}_prepared.csv'
    print(f"Loading data from: {data_path}")
    df = pd.read_csv(data_path)
    print(f"✓ Loaded {len(df):,} rows\n")
    
    # Separate features and targets
    target_col = 'target' if 'target' in df.columns else 'label'
    feature_cols = [col for col in df.columns if col not in [target_col, 'timestamp', 'date', 'time']]
    features = df[feature_cols].values
    targets = df[target_col].values
    
    # Split data
    n_samples = len(features)
    train_end = int(n_samples * config['train_ratio'])
    val_end = int(n_samples * (config['train_ratio'] + config['val_ratio']))
    
    train_features, train_targets = features[:train_end], targets[:train_end]
    val_features, val_targets = features[train_end:val_end], targets[train_end:val_end]
    test_features, test_targets = features[val_end:], targets[val_end:]
    
    print(f"Dataset split:")
    print(f"  Train: {len(train_features):,} samples ({config['train_ratio']*100:.0f}%)")
    print(f"  Val:   {len(val_features):,} samples ({config['val_ratio']*100:.0f}%)")
    print(f"  Test:  {len(test_features):,} samples ({config['test_ratio']*100:.0f}%)\n")
    
    # Create datasets
    train_dataset = SequenceDataset(train_features, train_targets, T_IN)
    val_dataset = SequenceDataset(val_features, val_targets, T_IN)
    test_dataset = SequenceDataset(test_features, test_targets, T_IN)
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)
    
    # Create model
    model_config = ModelConfig(
        num_features=len(feature_cols),
        hidden_size_lstm=config['hidden_size_lstm'],
        num_layers_lstm=config['num_layers_lstm'],
        cnn_num_filters=config['cnn_num_filters'],
        attention_dim=config['attention_dim'],
        dropout=config['dropout'],
        num_classes=config['num_classes'],
        use_optimized_attention=False
    )
    
    model = HybridCNNLSTMAttention(model_config)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, device = setup_multi_gpu(model, device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model: HybridCNNLSTMAttention ({total_params:,} parameters)")
    print(f"Device: {device}\n")
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=config['lr_scheduler_factor'], 
        patience=config['lr_scheduler_patience'], verbose=False
    )
    scaler = torch.cuda.amp.GradScaler() if config['use_amp'] and torch.cuda.is_available() else None
    
    # Checkpoint directory
    checkpoint_dir = ROOT / config['checkpoint_dir']
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    checkpoint_path = checkpoint_dir / f'{pair}_best_model.pt'
    
    # Training loop
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
    best_val_loss = float('inf')
    patience_counter = 0
    
    print(f"Starting training for {config['epochs']} epochs...\n")
    
    for epoch in range(1, config['epochs'] + 1):
        train_loss, train_acc = train_epoch_supervised(
            model, train_loader, criterion, optimizer, device, scaler, config['grad_clip']
        )
        val_loss, val_acc = validate_supervised(model, val_loader, criterion, device)
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"Epoch {epoch:3d}/{config['epochs']} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.2f}% | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.2f}% | "
                  f"LR: {current_lr:.2e}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': get_unwrapped_model(model).state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_acc': val_acc,
                'model_config': model_config.__dict__,
                'pair': pair,
                'history': history
            }, checkpoint_path)
        else:
            patience_counter += 1
        
        if patience_counter >= config['early_stop_patience']:
            print(f"\nEarly stopping at epoch {epoch}")
            break
    
    # Test evaluation
    checkpoint = torch.load(checkpoint_path)
    get_unwrapped_model(model).load_state_dict(checkpoint['model_state_dict'])
    test_loss, test_acc = validate_supervised(model, test_loader, criterion, device)
    
    elapsed_time = time.time() - start_time
    
    print(f"\n{'='*80}")
    print(f"TRAINING COMPLETE: {pair.upper()}")
    print(f"{'='*80}")
    print(f"Time elapsed: {format_time(elapsed_time)}")
    print(f"Best val loss: {best_val_loss:.4f} (epoch {checkpoint['epoch']})")
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_acc:.2f}%")
    print(f"Checkpoint: {checkpoint_path}")
    print(f"{'='*80}\n")
    
    return {
        'pair': pair,
        'best_val_loss': best_val_loss,
        'test_loss': test_loss,
        'test_acc': test_acc,
        'epochs_trained': len(history['train_loss']),
        'time_elapsed': elapsed_time,
        'checkpoint_path': str(checkpoint_path)
    }

### 12. ***SAC (Reinforcement Learning) Functions***

In [ ]:
def evaluate_sac_agent(agent, env, num_episodes=10):
    """Evaluate SAC agent over multiple episodes."""
    episode_rewards = []
    episode_lengths = []
    
    for _ in range(num_episodes):
        obs = env.reset()
        episode_reward = 0
        episode_length = 0
        done = False
        
        while not done:
            # Get state vector from observation
            state = np.array([obs['portfolio_value'], obs['position'], obs['cash']])
            action = agent.select_action(state, evaluate=True)
            
            # Convert continuous action to OrderAction
            size = abs(action[0]) * 10.0  # Scale action
            side = 'buy' if action[0] > 0 else 'sell'
            order = OrderAction(action_type='market', side=side, size=size)
            
            obs, reward, done, info = env.step(order)
            episode_reward += reward
            episode_length += 1
        
        episode_rewards.append(episode_reward)
        episode_lengths.append(episode_length)
    
    return {
        'mean_reward': np.mean(episode_rewards),
        'std_reward': np.std(episode_rewards),
        'mean_length': np.mean(episode_lengths)
    }


def train_sac(pair, config):
    """Complete SAC training pipeline for a single pair."""
    print(f"\n{'='*80}")
    print(f"SAC TRAINING: {pair.upper()}")
    print(f"{'='*80}\n")
    
    import time
    start_time = time.time()
    
    # Load data for environment
    data_path = ROOT / 'data' / 'data' / pair / f'{pair}_prepared.csv'
    print(f"Loading data from: {data_path}")
    df = pd.read_csv(data_path)
    print(f"✓ Loaded {len(df):,} rows\n")
    
    # Create execution environment
    exec_config = ExecutionConfig(
        initial_cash=config['initial_cash'],
        time_horizon=config['time_horizon'],
        commission_pct=config['commission_pct'],
        spread_bps=config['spread_bps'],
        reward_type=config['reward_type'],
        reward_scaling=config['reward_scaling']
    )
    
    env = SimulatedRetailExecutionEnv(exec_config, seed=42)
    print(f"Environment: SimulatedRetailExecutionEnv")
    print(f"  Initial cash: ${config['initial_cash']:,.2f}")
    print(f"  Time horizon: {config['time_horizon']} steps")
    print(f"  Reward type: {config['reward_type']}\n")
    
    # Create SAC agent
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    state_dim = 3  # portfolio_value, position, cash
    action_dim = 1  # continuous action (trade size)
    
    agent = SACAgent(
        state_dim=state_dim,
        action_dim=action_dim,
        hidden_dim=config['hidden_dim'],
        lr=config['learning_rate'],
        gamma=config['gamma'],
        tau=config['tau'],
        alpha=config['alpha'],
        auto_entropy_tuning=config['auto_entropy_tuning'],
        device=device
    )
    
    print(f"SAC Agent created (device: {device})")
    print(f"  State dim: {state_dim}")
    print(f"  Action dim: {action_dim}")
    print(f"  Hidden dim: {config['hidden_dim']}")
    print(f"  Auto entropy tuning: {config['auto_entropy_tuning']}\n")
    
    # Replay buffer
    replay_buffer = ReplayBuffer(capacity=config['replay_buffer_size'])
    
    # Checkpoint directory
    checkpoint_dir = ROOT / config['checkpoint_dir']
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    # Training loop
    print(f"Starting SAC training for {config['total_steps']:,} steps...\n")
    
    obs = env.reset()
    episode_reward = 0
    episode_length = 0
    episode_num = 0
    
    episode_rewards = []
    update_metrics = []
    
    for step in tqdm(range(1, config['total_steps'] + 1), desc="Training"):
        # Get state
        state = np.array([obs['portfolio_value'], obs['position'], obs['cash']])
        
        # Select action
        if step < config['warmup_steps']:
            action = np.random.uniform(-1, 1, size=(action_dim,))
        else:
            action = agent.select_action(state, evaluate=False)
        
        # Convert to OrderAction
        size = abs(action[0]) * 10.0
        side = 'buy' if action[0] > 0 else 'sell'
        order = OrderAction(action_type='market', side=side, size=size)
        
        # Step environment
        next_obs, reward, done, info = env.step(order)
        next_state = np.array([next_obs['portfolio_value'], next_obs['position'], next_obs['cash']])
        
        # Store transition
        replay_buffer.add(state, action[0], reward, next_state, done)
        
        episode_reward += reward
        episode_length += 1
        obs = next_obs
        
        # Update agent
        if len(replay_buffer) >= config['batch_size'] and step % config['update_interval'] == 0:
            metrics = agent.update(replay_buffer, batch_size=config['batch_size'])
            update_metrics.append(metrics)
        
        # Episode end
        if done:
            episode_rewards.append(episode_reward)
            episode_num += 1
            obs = env.reset()
            episode_reward = 0
            episode_length = 0
        
        # Evaluation
        if step % config['eval_interval'] == 0:
            eval_results = evaluate_sac_agent(agent, env, num_episodes=config['eval_episodes'])
            print(f"\nStep {step:6d} | "
                  f"Episodes: {episode_num} | "
                  f"Eval reward: {eval_results['mean_reward']:.2f} ± {eval_results['std_reward']:.2f} | "
                  f"Alpha: {agent.alpha:.3f}")
        
        # Save checkpoint
        if step % config['save_interval'] == 0:
            checkpoint_path = checkpoint_dir / f'{pair}_sac_step_{step}.pt'
            agent.save(str(checkpoint_path))
    
    # Final evaluation
    final_eval = evaluate_sac_agent(agent, env, num_episodes=50)
    
    # Save final checkpoint
    final_checkpoint = checkpoint_dir / f'{pair}_sac_final.pt'
    agent.save(str(final_checkpoint))
    
    elapsed_time = time.time() - start_time
    
    print(f"\n{'='*80}")
    print(f"SAC TRAINING COMPLETE: {pair.upper()}")
    print(f"{'='*80}")
    print(f"Time elapsed: {format_time(elapsed_time)}")
    print(f"Total episodes: {episode_num}")
    print(f"Final eval reward: {final_eval['mean_reward']:.2f} ± {final_eval['std_reward']:.2f}")
    print(f"Checkpoint: {final_checkpoint}")
    print(f"{'='*80}\n")
    
    return {
        'pair': pair,
        'total_episodes': episode_num,
        'final_eval_reward_mean': final_eval['mean_reward'],
        'final_eval_reward_std': final_eval['std_reward'],
        'time_elapsed': elapsed_time,
        'checkpoint_path': str(final_checkpoint)
    }

## 13. ***Multi-Pair Training Loop***

In [ ]:
# Track results for all pairs
all_results = []

print(f"\n{'#'*80}")
print(f"MULTI-PAIR TRAINING SESSION")
print(f"{'#'*80}")
print(f"Mode: {TRAINING_MODE.upper()}")
print(f"Pairs: {', '.join([p.upper() for p in PAIRS])}")
print(f"Total pairs: {len(PAIRS)}")
print(f"{'#'*80}\n")

import time
session_start_time = time.time()

for idx, pair in enumerate(PAIRS, 1):
    print(f"\n[{idx}/{len(PAIRS)}] Processing {pair.upper()}...")
    
    try:
        if TRAINING_MODE == 'supervised':
            result = train_supervised(pair, SUPERVISED_CONFIG)
        elif TRAINING_MODE == 'sac':
            result = train_sac(pair, SAC_CONFIG)
        else:
            raise ValueError(f"Unknown training mode: {TRAINING_MODE}")
        
        all_results.append(result)
        print(f"✓ {pair.upper()} completed successfully")
        
    except Exception as e:
        print(f"✗ {pair.upper()} failed: {e}")
        all_results.append({
            'pair': pair,
            'error': str(e)
        })
    
    # Show remaining pairs
    if idx < len(PAIRS):
        remaining = PAIRS[idx:]
        print(f"\nRemaining pairs: {', '.join([p.upper() for p in remaining])}")

session_elapsed_time = time.time() - session_start_time

print(f"\n{'#'*80}")
print(f"ALL TRAINING COMPLETE")
print(f"{'#'*80}")
print(f"Total session time: {format_time(session_elapsed_time)}")
print(f"Pairs processed: {len(all_results)}/{len(PAIRS)}")
print(f"{'#'*80}\n")

## 8. Visualization

In [ ]:
if TRAINING_MODE == 'supervised':
    # Plot test accuracy comparison
    successful_results = [r for r in all_results if 'test_acc' in r]
    
    if successful_results:
        pairs = [r['pair'].upper() for r in successful_results]
        test_accs = [r['test_acc'] for r in successful_results]
        times = [r['time_elapsed'] / 60 for r in successful_results]  # Convert to minutes
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Test accuracy
        axes[0].bar(pairs, test_accs, color='steelblue', alpha=0.8)
        axes[0].set_ylabel('Test Accuracy (%)')
        axes[0].set_title('Test Accuracy by Pair')
        axes[0].grid(True, alpha=0.3, axis='y')
        axes[0].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Random baseline')
        axes[0].legend()
        
        # Training time
        axes[1].bar(pairs, times, color='coral', alpha=0.8)
        axes[1].set_ylabel('Training Time (minutes)')
        axes[1].set_title('Training Time by Pair')
        axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()

elif TRAINING_MODE == 'sac':
    # Plot final reward comparison
    successful_results = [r for r in all_results if 'final_eval_reward_mean' in r]
    
    if successful_results:
        pairs = [r['pair'].upper() for r in successful_results]
        rewards = [r['final_eval_reward_mean'] for r in successful_results]
        stds = [r['final_eval_reward_std'] for r in successful_results]
        times = [r['time_elapsed'] / 60 for r in successful_results]
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Final rewards
        axes[0].bar(pairs, rewards, yerr=stds, color='mediumseagreen', alpha=0.8, capsize=5)
        axes[0].set_ylabel('Final Eval Reward')
        axes[0].set_title('Final Evaluation Reward by Pair')
        axes[0].grid(True, alpha=0.3, axis='y')
        axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
        
        # Training time
        axes[1].bar(pairs, times, color='coral', alpha=0.8)
        axes[1].set_ylabel('Training Time (minutes)')
        axes[1].set_title('Training Time by Pair')
        axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()

## 14. ***BACKUP TO DRIVE***

In [ ]:
# Save checkpoints to Google Drive for persistence
import shutil

drive_checkpoint_dir = Path('/content/drive/MyDrive/Sequence_Models')
drive_checkpoint_dir.mkdir(parents=True, exist_ok=True)

print("Backing up checkpoints to Google Drive...\\n")

checkpoint_dir = ROOT / 'models' / 'checkpoints'
for checkpoint_file in checkpoint_dir.glob('*.pt'):
    shutil.copy2(checkpoint_file, drive_checkpoint_dir / checkpoint_file.name)
    print(f"✓ {checkpoint_file.name}")

print(f"\\n✓ Checkpoints saved to: {drive_checkpoint_dir}")